# Creating Collections as Data Using Federated Queries / Multiple Authors' Works

## About the Notebook
This notebook demonstrates **a streamlined pipeline for executing federated queries across multiple library Linked Open Data (LOD) endpoints**. It simplifies the process of retrieving and consolidating data about literary works and authors into a single dataset. The query outputs are merged into a final dataset that is exported as a CSV file for analyses or reuse.

The Spanish Golden Age ([Q530936](https://www.wikidata.org/wiki/Q530936), 1492-1659) was period of flourishing in arts and literature in Spain, coinciding with the political rise and decline of the Spanish Habsburg dynasty.


[![Launch](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/semanticnoodles/federated-cad/HEAD?filepath=notebooks/03-movement-multiple-authors/LOD-SpanishGoldenAge-Federated-CAD.ipynb)


<!--🔖 **How to Cite**: [![DOI](https://zenodo.org/badge/DOI_NUMBER.svg)](https://doi.org/DOI_NUMBER) fix once we get the CITATION.cff set up in the GitHub repo-->

---

### Core Modules on Use

- `SPARQLWrapper` is used to interact with SPARQL endpoints and execute queries to retrieve data.
- `pandas` essential to perform data handling and manipulation tasks.
- `aiohttp` and `nest_asyncio` enable asynchronous programming which is crucial for efficient data retrieval by allowing multiple tasks to run concurrently. In particular, `nest_asyncio` is important in a Jupyter notebook for running async tasks.
- `tqdm` is used to display progress bars for long-running tasks.

In [34]:
# Core modules
import asyncio
import nest_asyncio
from SPARQLWrapper import SPARQLWrapper, JSON

# Data handling and manipulation
import pandas as pd

# Network and HTTP requests
import aiohttp

# Utility and support
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio
import time
import random

## Queries General Parameters Setup

We start by storing into `endpoint_url` the address of the Wikidata endpoint.

Then, a function called `execute_query` is defined: it takes a SPARQL query and the endpoint address as input. Next, it sends the query to the Wikidata endpoint, waits for a response (specifying a timeout in case of complex queries), and formats the received data into a list of dictionaries, which it then returns as the final output.

In [ ]:
# Define the SPARQL endpoint on use (outside the function)
endpoint_url = "https://query.wikidata.org/sparql"

# Function to execute a query and return results as a list of dictionaries
def execute_query(query, endpoint_url):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setTimeout(180)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    sparql.addCustomHttpHeader(
        "User-Agent", "LODCaD/1.0 (email@gmail.com)"
    )  # Replace with your bot's details
    results = sparql.query().convert()
    return results["results"]["bindings"]

Before laying out our queries we need to identify the Wikidata ids fo the authors from the Spanish Golden Age (Wikidata ID: Q530936) who are represented to some extent across our selected repositories. To perform an environmental scan, we retrieve all authors related to the Spanish Golden Age using a SPARQL query stored in the `fetch_all_authors` variable to be used for the following LOD repositories:

- `BNE`: Biblioteca Nacional de España (The Spanish National Library).
- `BNF`: Bibliothèque nationale de France (The National Library of France).
- `BVMC`: Biblioteca Virtual Miguel de Cervantes (The Miguel de Cervantes Virtual Library).

In [36]:
fetch_all_authors = """
SELECT DISTINCT ?author ?authorLabel ?bneID ?idBnF ?bvmcID WHERE {
  ?author wdt:P135 wd:Q530936;  # Spanish Golden Age
          rdfs:label ?authorLabel.
  OPTIONAL { ?author wdt:P950 ?bneID. }
  OPTIONAL { ?author wdt:P268 ?idBnF. }
  OPTIONAL { ?author wdt:P2799 ?bvmcID. }
  FILTER(LANG(?authorLabel) = "en")
}
LIMIT 200
"""

Once the query has been defined, the function `query_to_dataframe` is created to run it, process the results into a structured format, and store them in a pandas dataframe for further analysis.

In [37]:
def query_to_dataframe(query):
    # Fetch results from Wikidata
    results = execute_query(query, endpoint_url)

    # Map results to a list of dictionaries
    data = []
    for result in results:
        row = {
            "author": result.get("author", {}).get("value", ""),
            "authorLabel": result.get("authorLabel", {}).get("value", ""),
            "bneID": result.get("bneID", {}).get("value", ""),
            "idBnF": result.get("idBnF", {}).get("value", ""),
            "bvmcID": result.get("bvmcID", {}).get("value", ""),
        }
        data.append(row)

    # Convert to a dataframe
    return pd.DataFrame(data)

In [38]:
# Get a dataframe with all the authors
df_all_authors = query_to_dataframe(fetch_all_authors)

# Preview
df_all_authors

,author,authorLabel,bneID,idBnF,bvmcID
0,http://www.wikidata.org/entity/Q5682,Miguel de Cervantes,XX1718747,118957747,40
1,http://www.wikidata.org/entity/Q94802,The Burial of the Count of Orgaz,,,
2,http://www.wikidata.org/entity/Q165257,Lope de Vega,XX1719671,11927819k,72
3,http://www.wikidata.org/entity/Q201315,Francisco Quevedo,XX1066651,118873287,6
4,http://www.wikidata.org/entity/Q2450139,Boys Eating Grapes and Melon,,,
5,http://www.wikidata.org/entity/Q2905689,María de Zayas,XX913942,119878263,105
6,http://www.wikidata.org/entity/Q5551713,Jerónimo de Pasamonte,XX1183253,,6748
7,http://www.wikidata.org/entity/Q3893270,Alonso Jerónimo de Salas Barbadillo,XX1054399,120545809,895
8,http://www.wikidata.org/entity/Q5723718,Bautista Remiro de Navarra,XX1140785,144495928,
9,http://www.wikidata.org/entity/Q10304696,Jacinto Abad de Ayala,XX1171039,,


Subsequently, we decided to narrow down the list to include **only those authors who have identifiers in all three chosen LOD repositories** (BNE, BNF, and BVMC). To do so we created a second SPARQL query, `fetch_authors` and ran again `query_to_dataframe`.

In [39]:
fetch_authors = """
SELECT DISTINCT ?author ?authorLabel ?bneID ?idBnF ?bvmcID WHERE {
  ?author wdt:P950 ?bneID.
  ?author wdt:P268 ?idBnF.
  ?author wdt:P2799 ?bvmcID.
  ?author wdt:P135 wd:Q530936;
    rdfs:label ?authorLabel.
  FILTER(LANG(?authorLabel) = "en")
}
LIMIT 200
"""

# Get a dataframe with all the authors
df_authors = query_to_dataframe(fetch_authors)

# Preview
df_authors

,author,authorLabel,bneID,idBnF,bvmcID
0,http://www.wikidata.org/entity/Q5682,Miguel de Cervantes,XX1718747,118957747,40
1,http://www.wikidata.org/entity/Q165257,Lope de Vega,XX1719671,11927819k,72
2,http://www.wikidata.org/entity/Q201315,Francisco Quevedo,XX1066651,118873287,6
3,http://www.wikidata.org/entity/Q2905689,María de Zayas,XX913942,119878263,105
4,http://www.wikidata.org/entity/Q3893270,Alonso Jerónimo de Salas Barbadillo,XX1054399,120545809,895


 As last step, we clean the Wikidata ids for our authors (keeping just the tail of the URLs), and store them in a new list called `wikidata_ids` for further processing.

 Now we have all the required information to craft the queries.

In [66]:
# Get a list of raw Wikidata ids (URLs) from the author column
wikidata_raw_ids = df_authors["author"].tolist()

# Extract Wikidata ids from the URLs by splitting them at '/' and taking the last element
wikidata_ids = [url.split("/")[-1] for url in wikidata_raw_ids]

# Preview
wikidata_ids

['Q5682', 'Q165257', 'Q201315', 'Q2905689', 'Q3893270']

## Functions Design

This section focuses on creating reusable functions to efficiently query and retrieve data from LOD repositories, leveraging the use of query templates as outlined in the Queries run.

In short `fetch_paginated_results_by_author` orchestrates the process, calling `fetch_results_for_author for each author`, which in turn calls `fetch_query` to retrieve the actual data. This function chaining ensures that data is fetched efficiently and facilitates handling potential issues during the process.

Additionally, we defined `track_progress`, **a function to provide a visual progress bar using the `tqdm` library**, and therefore support tracking the execution of the asynchronous tasks.

In [41]:
# Enable progression bar display with tqdm
async def track_progress(tasks, desc="Processing"):
    results = []
    for coro in tqdm_asyncio.as_completed(tasks, desc=desc, total=len(tasks)):
        result = await coro
        results.append(result)
    return results

Then, the function `fect_query` is designed to send a SPARQL query to a specified endpoint and handle potential errors or rate limiting.

In [42]:
## Define a function to ship the query to a specified endpoint
async def fetch_query(session, query, endpoint_url, timeout, retries):
    headers = {"Accept": "application/sparql-results+json"}

    for attempt in range(retries):
        try:
            print(f"[Attempt {attempt + 1}/{retries}] Querying {endpoint_url}")
            print(f"→ Query Preview: {query[:200].strip()}...\n")

            async with session.post(
                endpoint_url, data={"query": query}, headers=headers, timeout=timeout
            ) as response:
                if response.status == 200:
                    print(
                        "✅ Query successful.\n"
                    )  # added some emoji to enhance interpretability
                    return await response.json()
                elif response.status == 429:
                    wait_time = 2**attempt + random.uniform(0, 1)
                    print(
                        f"⚠️ 429 Too Many Requests — Retrying in {wait_time:.2f} seconds...\n"
                    )
                    await asyncio.sleep(wait_time)
                else:
                    print(f"❌ Unexpected status {response.status}. Raising...\n")
                    response.raise_for_status()

        except aiohttp.ClientError as e:
            print(f"🚨 Client error on attempt {attempt + 1}: {e}\n")
        except asyncio.TimeoutError:
            print(f"⏳ Timeout on attempt {attempt + 1}\n")
        except Exception as e:
            print(f"❗ General error on attempt {attempt + 1}: {e}\n")

        if attempt == retries - 1:
            print(f"❌ Failed after {retries} attempts. Giving up.\n")
            return None

 To complement and refine `fetch_query`, **the function `fetch_results_for_author` is designed to retrieve data for a specific author from a given endpoint**. It is built to handle scenarios where the data is split across multiple pages (pagination) and potential network issues.

In [43]:
# Define a function to get results for a specified author
async def fetch_results_for_author(
    session, wikidata_id, query_template, endpoint_url, timeout, limit, max_retries
):
    print(f"\n🔍 Fetching results for author: {wikidata_id}")
    author_query = query_template.replace("<WIKIDATA_ID>", wikidata_id)
    offset = 0
    author_results = []

    while True:
        paginated_query = author_query.replace(
            "LIMIT <LIMIT>", f"LIMIT {limit} OFFSET {offset}"
        )
        print(f"  → OFFSET {offset}, LIMIT {limit}")

        result = await fetch_query(
            session, paginated_query, endpoint_url, timeout, max_retries
        )

        if result:
            bindings = result.get("results", {}).get("bindings", [])
            if not bindings:
                print(f"  ✅ No more results for {wikidata_id}. Done.\n")
                break
            author_results.extend(bindings)
            offset += limit
        else:
            print(f"  ❌ Failed to fetch data for author {wikidata_id}. Skipping.\n")
            break

    return author_results

Last, the function **`fetch_paginated_results_by_author` streamlines the process of retrieving data about multiple authors**, acting as a sort of wrapper for the previously specified functions.

In [44]:
# Define the wrapper function to handle data from multiple authors
async def fetch_paginated_results_by_author(
    wikidata_ids, query_template, endpoint_url, timeout=10, limit=100, max_retries=5
):
    print(f"🚀 Starting queries for {len(wikidata_ids)} authors...\n")

    async with aiohttp.ClientSession() as session:
        tasks = [
            fetch_results_for_author(
                session,
                wikidata_id,
                query_template,
                endpoint_url,
                timeout,
                limit,
                max_retries,
            )
            for wikidata_id in tqdm(wikidata_ids)
        ]

        results = await asyncio.gather(*tasks)

        results_dict = {
            wikidata_id: result for wikidata_id, result in zip(wikidata_ids, results)
        }
        print("\n📊 Summary of records fetched per author:")
        for author_id, records in results_dict.items():
            print(f"  {author_id}: {len(records)} records")
        print()

        return results_dict

## Queries Design

Here the actual SPARQL queries are defined and executed to retrieve data from our chosen LOD repositories. The tasks are run using the previously defined functions to handle the query execution and data retrieval process.

In [45]:
nest_asyncio.apply() # required to run multiple queries simultaneously (parallelization) within the Jupyter Notebook.

**The queries are designed with placeholders to make them reusable and adaptable for different authors and better handle different data collection limits.** If you look closely at each of the SPARQL queries structure, you will notice the use of two placeholders:

- `<WIKIDATA_ID>`. This placeholder is intended to be replaced with the actual Wikidata id of a specific author, specifically the ones we stored in the `wikidata_ids` list.  
- `<LIMIT>`. This placeholder controls the maximum number of results returned by the query. `<LIMIT>` is intended to be replaced with an actual numerical value, specifying the desired number of results to obtain during each iteration.

### BNE

In [46]:
# Define BNE query
bne_query = """
PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode
        WHERE {
            wd:<WIKIDATA_ID> wdt:P950 ?id .
            wd:<WIKIDATA_ID> rdfs:label ?author.  FILTER(LANG(?author) = "en").
            BIND(uri(concat("https://datos.bne.es/resource/", ?id)) as ?bneID)
            SERVICE <http://datos.bne.es/sparql> {
                ?bneID bne-def:OP5001 ?work .
                ?work rdfs:label ?workLabel .
                OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3003 ?placeOfProduction}
                OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3006 ?yearOfPublication}
                OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition dcterms:language ?langCode}
            }
        }
        LIMIT <LIMIT>
"""

In [47]:
# Run the asynchronous function
bne_combined_results = asyncio.run(
    fetch_paginated_results_by_author(wikidata_ids, bne_query, endpoint_url)
)

# Flatten the results and convert to dataframe
bne_data = []
for author_id, results in bne_combined_results.items():
    for result in results:
        # Extract relevant fields from the SPARQL query result
        author = result.get("author", {}).get("value", None)
        # author_label = result.get('authorLabel', {}).get('value', None)
        work = result.get("work", {}).get("value", None)
        work_label = result.get("workLabel", {}).get("value", None)
        edition = result.get("edition", {}).get("value", None)
        place_of_production = result.get("placeOfProduction", {}).get("value", None)
        year_of_publication = result.get("yearOfPublication", {}).get("value", None)
        lang_code = result.get("langCode", {}).get("value", None)

        bne_data.append(
            {
                "Source": "BNE",  # specifies BNE as source
                "Author": author,
                #'Author Label': author_label,
                "Work": work,
                "Work Label": work_label,
                "Edition": edition,
                "Place of Production": place_of_production,
                "Year of Publication": year_of_publication,
                "Language Code": lang_code,
            }
        )

🚀 Starting queries for 5 authors...



100%|██████████| 5/5 [00:00<00:00, 161319.38it/s]


🔍 Fetching results for author: Q5682
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode...


🔍 Fetching results for author: Q165257
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode...


🔍 Fetching results for author: Q201315
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCod

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode...

✅ Query successful.

  ✅ No more results for Q2905689. Done.

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode...

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX bne-def: <https://datos.bne.es/def/>
        PREFIX dcterms: <http://purl.org/dc/terms/>

        SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublica

In [48]:
# Render bne_data into a dataframe
df_bne = pd.DataFrame(bne_data)

# Preview
df_bne.head()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
0,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383764,Novelas ejemplares,https://datos.bne.es/resource/bima0000013178,Paris,1809,http://id.loc.gov/vocabulary/languages/fre
1,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383563,Don Quijote de la Mancha,https://datos.bne.es/resource/bimo0000398030,[Murcia,D.L. 1993,http://id.loc.gov/vocabulary/languages/spa
2,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX1924290,La cueva de Salamanca,https://datos.bne.es/resource/bimo0002046013,[Granada],2005,http://id.loc.gov/vocabulary/languages/spa
3,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383764,Novelas ejemplares,https://datos.bne.es/resource/Mimo0001661915,[Tokio,1993],http://id.loc.gov/vocabulary/languages/jpn
4,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX4894754,El celoso extremeño,https://datos.bne.es/resource/a5599310,[S.l],[1917?],http://id.loc.gov/vocabulary/languages/rus


In [49]:
df_bne.describe()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
count,2959,2959,2959,2959,2608,2487,2600,2584
unique,1,5,823,814,2308,367,689,24
top,BNE,Lope de Vega,https://datos.bne.es/resource/XX3383564,Historia de la vida del Buscón,https://datos.bne.es/resource/a6971660,Madrid,S.XVII,http://id.loc.gov/vocabulary/languages/spa
freq,2959,1862,232,232,8,681,50,2337


### BNF

In [50]:
bnf_query = """
  PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?author ?expression ?title ?edition ?placeOfPublication ?yearOfPublication ?langCode WHERE {
            wd:<WIKIDATA_ID> wdt:P268 ?id
            BIND(uri(concat(concat("http://data.bnf.fr/ark:/12148/cb", ?id),"#about")) as ?author)
            SERVICE <http://data.bnf.fr/sparql> {
                ?expression <http://id.loc.gov/vocabulary/relators/aut> ?author .
                OPTIONAL {?expression dcterms:language ?langCode .}
                OPTIONAL {?expression dcterms:publisher ?edition .}
                ?manifestation rdarelationships:expressionManifested ?expression .
                ?manifestation dcterms:title ?title .
                ?manifestation dcterms:date ?yearOfPublication .
                OPTIONAL{ ?manifestation rdagroup1elements:placeOfPublication ?placeOfPublication .}
            }
        }
        LIMIT <LIMIT>
"""

In [51]:
# Run the asynchronous function
bnf_combined_results = asyncio.run(
    fetch_paginated_results_by_author(wikidata_ids, bnf_query, endpoint_url, timeout=30)
)

# Flatten the results and convert to DataFrame
bnf_data = []
for author_id, results in bnf_combined_results.items():
    for result in results:
        # Extract relevant fields from the SPARQL query result
        author = result.get("author", {}).get("value", "")
        work = result.get("expression", {}).get("value", "") # RDA requirement
        work_label = result.get("title", {}).get("value", "") # we nest the work title here
        edition = result.get("edition", {}).get("value", "")
        place_of_production = result.get("placeOfPublication", {}).get("value", "") # here it takes Publication instead of Production
        year_of_publication = result.get("yearOfPublication", {}).get("value", "")
        lang_code = result.get("langCode", {}).get("value", "")

        bnf_data.append(
            {
                "Source": "BNF",  # specifies BNF as source
                "Author": author,
                "Work": work,
                "Work Label": work_label,
                "Edition": edition,
                "Place of Production": place_of_production,
                "Year of Publication": year_of_publication,
                "Language Code": lang_code,
            }
        )

🚀 Starting queries for 5 authors...



100%|██████████| 5/5 [00:00<00:00, 142663.40it/s]


🔍 Fetching results for author: Q5682
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?autho...


🔍 Fetching results for author: Q165257
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?autho...


🔍 Fetching results for author: Q201315
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?autho...

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?autho...

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
  PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

        SELECT ?autho...

✅ Query successful.

  ✅ No mor

In [52]:
# Create DataFrame
df_bnf = pd.DataFrame(bnf_data)

# Flatten the results and get to dataframe
df_bnf.head()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
0,BNF,http://data.bnf.fr/ark:/12148/cb118957747#about,http://data.bnf.fr/ark:/12148/cb39230109z#Expr...,Novelas ejemplares,,Barcelona,2001,http://id.loc.gov/vocabulary/iso639-2/spa
1,BNF,http://data.bnf.fr/ark:/12148/cb118957747#about,http://data.bnf.fr/ark:/12148/cb42085217w#Expr...,"Michaī́l Tservántes, O epitī́deios eugenī́...",,Athī́na,2007,http://id.loc.gov/vocabulary/iso639-2/gre
2,BNF,http://data.bnf.fr/ark:/12148/cb118957747#about,http://data.bnf.fr/ark:/12148/cb331009980#Expr...,Vu du pont,,Paris,1959,http://id.loc.gov/vocabulary/iso639-2/fre
3,BNF,http://data.bnf.fr/ark:/12148/cb118957747#about,http://data.bnf.fr/ark:/12148/cb32945636w#Expr...,La Danse du château,,[Paris],1962,http://id.loc.gov/vocabulary/iso639-2/fre
4,BNF,http://data.bnf.fr/ark:/12148/cb118957747#about,http://data.bnf.fr/ark:/12148/cb30213156j#Expr...,Histoire de l'admirable Don Quichotte de la Ma...,,Paris,1862,http://id.loc.gov/vocabulary/iso639-2/fre


In [53]:
# Preview
df_bnf.describe()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
count,2085,2085,2085,2085,2085,2085,2085,2085
unique,1,5,1699,1281,1,394,448,24
top,BNF,http://data.bnf.fr/ark:/12148/cb118957747#about,http://data.bnf.fr/ark:/12148/cb427142507#Expr...,Don Quichotte,,Paris,1993,http://id.loc.gov/vocabulary/iso639-2/spa
freq,2085,1096,4,46,2085,538,29,1037


Apparently, we cannot not get any info related to edition.

### BVMC

In [54]:
bvmc_query = """
PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>

        SELECT ?author ?work ?workLabel ?placeOfProduction ?yearOfPublication ?langCode
        WHERE {
            wd:<WIKIDATA_ID> wdt:P2799 ?id .
            wd:<WIKIDATA_ID> rdfs:label ?author.  FILTER(LANG(?author) = "en").
            BIND(uri(concat("https://data.cervantesvirtual.com/person/", ?id)) as ?bvmcID)
            SERVICE <http://data.cervantesvirtual.com/openrdf-sesame/repositories/data> {
                ?work rdaw:author ?bvmcID .
                ?work rdfs:label ?workLabel .
                ?work rdaw:manifestationOfWork ?manifestation .
                ?work rdaw:expressionOfWork ?expression .
                OPTIONAL {?expression rdae:languageOfExpression ?language . ?language madsrdf:code ?langCode .}
                OPTIONAL {?manifestation rdam:placeOfProduction ?placeOfProduction .}
                OPTIONAL {?manifestation rdam:dateOfPublication ?dateOfPublication . BIND(REPLACE(str(?dateOfPublication), "https://data.cervantesvirtual.com/date/", "", "i") AS ?yearOfPublication) .}
            }
        }
        LIMIT <LIMIT>
"""

In [55]:
# Run the asynchronous function
bvmc_combined_results = asyncio.run(
    fetch_paginated_results_by_author(wikidata_ids, bvmc_query, endpoint_url)
)

# Flatten the results and convert to DataFrame
bvmc_data = []
for author_id, results in bvmc_combined_results.items():
    for result in results:
        # Extract relevant fields from the SPARQL query result
        author = result.get("author", {}).get("value", "")
        work = result.get("work", {}).get("value", "")
        work_label = result.get("workLabel", {}).get("value", "")
        place_of_production = result.get("placeOfProduction", {}).get("value", "")
        year_of_publication = result.get("yearOfPublication", {}).get("value", "")
        lang_code = result.get("langCode", {}).get("value", "")

        bvmc_data.append(
            {
                "Source": "BVMC",
                "Author": author,
                "Work": work,
                "Work Label": work_label,
                "Place of Production": place_of_production,
                "Year of Publication": year_of_publication,
                "Language Code": lang_code,
            }
        )

🚀 Starting queries for 5 authors...



100%|██████████| 5/5 [00:00<00:00, 122640.47it/s]


🔍 Fetching results for author: Q5682
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http:...


🔍 Fetching results for author: Q165257
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http:...


🔍 Fetching results for author: Q201315
  → OFFSET 0, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PRE

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http:...

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http:...

✅ Query successful.

  → OFFSET 100, LIMIT 100
[Attempt 1/5] Querying https://query.wikidata.org/sparql
→ Query Preview: PREFIX rdaw: <http://rdaregistry.info/Elements/w/>
        PREFIX rdam: <http://rdaregistry.info/Elements/m/>
        PREFIX rdae: <http://rdaregistry.info/Elements/e/>
        PREFIX madsrdf: <http:...

✅ Query successful.

  ✅ 

In [56]:
# Create DataFrame
df_bvmc = pd.DataFrame(bvmc_data)

# Flatten the results and get to dataframe
df_bvmc.head()

,Source,Author,Work,Work Label,Place of Production,Year of Publication,Language Code
0,BVMC,Miguel de Cervantes,https://data.cervantesvirtual.com/work/1005253,Rinconete y Cortadillo,"Madrid : Revista de Archivos, Bibliotecas y Mu...",1920,es
1,BVMC,Miguel de Cervantes,https://data.cervantesvirtual.com/work/766798,Novelas ejemplares,"Barcelona : Salvador Ribas, 1881 (Establecimie...",1881,es
2,BVMC,Miguel de Cervantes,https://data.cervantesvirtual.com/work/770698,El ingenioso hidalgo Don Quijote de la Mancha,Barcelona : Centro editorial artístico de Migu...,1897,es
3,BVMC,Miguel de Cervantes,https://data.cervantesvirtual.com/work/21424,L'enginyós hidalgo Don Quixote de la Mancha,"Felanitx, Imprempta d'en Bartoméu Rèus, 1905-1906",1906,ca
4,BVMC,Miguel de Cervantes,https://data.cervantesvirtual.com/work/721923,El ingenioso hidalgo Don Quijote de la Mancha....,"Madrid, Ediciones de La Lectura, 1911-1913. Ca...",1911-1913,es


In [57]:
# Preview
df_bvmc.describe()

,Source,Author,Work,Work Label,Place of Production,Year of Publication,Language Code
count,1839,1839,1839,1839,1839,1839,1839
unique,1,5,1276,1204,465,214,8
top,BVMC,Lope de Vega,https://data.cervantesvirtual.com/work/2904,El ingenioso hidalgo Don Quijote de la Mancha,,,es
freq,1839,1124,108,110,600,462,1805


## Cumulative Dataset Creation

The Cumulative Dataset Creation section aims to combine the data retrieved from the BNE, BNF, and BVMC queries into a single, unified dataset, to be exported in .CSV format.

In [81]:
# Concatenate the obtained dataframes
df_combined = pd.concat([df_bne, df_bnf, df_bvmc], ignore_index=True)

In [82]:
# Fields summary stats
df_combined.describe()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
count,6883,6883,6883,6883,4693,6411,6524,6508
unique,3,10,3798,3018,2309,1165,929,55
top,BNE,Lope de Vega,https://datos.bne.es/resource/XX3383564,Historia de la vida del Buscón,,Madrid,,http://id.loc.gov/vocabulary/languages/spa
freq,2959,2986,232,236,2085,1128,462,2337


In [83]:
# Preview
df_combined.head()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
0,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383764,Novelas ejemplares,https://datos.bne.es/resource/bima0000013178,Paris,1809,http://id.loc.gov/vocabulary/languages/fre
1,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383563,Don Quijote de la Mancha,https://datos.bne.es/resource/bimo0000398030,[Murcia,D.L. 1993,http://id.loc.gov/vocabulary/languages/spa
2,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX1924290,La cueva de Salamanca,https://datos.bne.es/resource/bimo0002046013,[Granada],2005,http://id.loc.gov/vocabulary/languages/spa
3,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383764,Novelas ejemplares,https://datos.bne.es/resource/Mimo0001661915,[Tokio,1993],http://id.loc.gov/vocabulary/languages/jpn
4,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX4894754,El celoso extremeño,https://datos.bne.es/resource/a5599310,[S.l],[1917?],http://id.loc.gov/vocabulary/languages/rus


## Preliminary Data Cleaning
In this section, we demonstrate how to go through some preliminary data cleaning and transformation tasks. In particular, we focus on the following aspects:
- **Special characters removal**. We use ``str.replace()`` and ``str.removal`` methods to remove unwanted characters from the Place of Production and year of Publication columns.
- **Normalization of author names**. Different LOD repositories can store the information using different modalities. This may create inconsistencies as we observe in the case of the BNF, using URIs instead of human-readable authors' names. The  two codeblocks address the normalisation of the auhors' names. 

In [85]:
# Remove square brackets, parentheses, and extra whitespace from "Place of Production" and "Year of Publication"
df_combined["Place of Production"] = (
    df_combined["Place of Production"]
    .str.replace(r"[\[\]()]", "", regex=True)
    .str.strip()
)
df_combined["Year of Publication"] = (
    df_combined["Year of Publication"]
    .str.replace(r"[\[\]()]", "", regex=True)
    .str.strip()
)

# Preview the updated dataframe
df_combined.head()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
0,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383764,Novelas ejemplares,https://datos.bne.es/resource/bima0000013178,Paris,1809,http://id.loc.gov/vocabulary/languages/fre
1,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383563,Don Quijote de la Mancha,https://datos.bne.es/resource/bimo0000398030,Murcia,D.L. 1993,http://id.loc.gov/vocabulary/languages/spa
2,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX1924290,La cueva de Salamanca,https://datos.bne.es/resource/bimo0002046013,Granada,2005,http://id.loc.gov/vocabulary/languages/spa
3,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX3383764,Novelas ejemplares,https://datos.bne.es/resource/Mimo0001661915,Tokio,1993,http://id.loc.gov/vocabulary/languages/jpn
4,BNE,Miguel de Cervantes,https://datos.bne.es/resource/XX4894754,El celoso extremeño,https://datos.bne.es/resource/a5599310,S.l,1917?,http://id.loc.gov/vocabulary/languages/rus


In [86]:
# Author fields check
df_combined['Author'].unique()

array(['Miguel de Cervantes', 'Lope de Vega', 'Francisco Quevedo',
       'María de Zayas', 'Alonso Jerónimo de Salas Barbadillo',
       'http://data.bnf.fr/ark:/12148/cb118957747#about',
       'http://data.bnf.fr/ark:/12148/cb11927819k#about',
       'http://data.bnf.fr/ark:/12148/cb118873287#about',
       'http://data.bnf.fr/ark:/12148/cb119878263#about',
       'http://data.bnf.fr/ark:/12148/cb120545809#about'], dtype=object)

In [87]:
# Vocabulary to map author names vs. URIs
author_uri_to_name = {
    "http://data.bnf.fr/ark:/12148/cb118957747#about": "Miguel de Cervantes",
    "http://data.bnf.fr/ark:/12148/cb11927819k#about": "Lope de Vega",
    "http://data.bnf.fr/ark:/12148/cb118873287#about": "Francisco Quevedo",
    "http://data.bnf.fr/ark:/12148/cb119878263#about": "María de Zayas",
    "http://data.bnf.fr/ark:/12148/cb120545809#about": "Alonso Jerónimo de Salas Barbadillo",
}

In [88]:
# Define a reusable function to normalize author names
def normalize_authors(df, column, uri_name_map):
    return df.assign(**{column: df[column].replace(uri_name_map)})

In [89]:
# Normalize authors names
df_combined = normalize_authors(df_combined, "Author", author_uri_to_name)

# Preview
df_combined.describe()

,Source,Author,Work,Work Label,Edition,Place of Production,Year of Publication,Language Code
count,6883,6883,6883,6883,4693,6411,6524,6508
unique,3,5,3798,3018,2309,1083,763,55
top,BNE,Lope de Vega,https://datos.bne.es/resource/XX3383564,Historia de la vida del Buscón,,Madrid,,http://id.loc.gov/vocabulary/languages/spa
freq,2959,3631,232,236,2085,1265,462,2337


As we have normalized the authors' names and performed a pre-processing step, we now can export our dataset in .CSV format.

In [90]:
# Export the resulting dataframe in .CSV file format
df_combined.to_csv('df-golden-age.csv', index=False)